<a href="https://colab.research.google.com/github/ohjisu320/study_data_analytics/blob/main/docs/quests/CDAs/CDA_mixed_unVar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 전처리 과정

In [1]:
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 32 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (5,826 kB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 121747 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr

In [42]:
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import scipy.stats as stats
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

In [64]:
df_store = pd.read_csv('fruitstore_saleshistory.csv')

In [44]:
df_store.columns

Index(['공급일자', '요일', '공급월', '공급주차', '회원번호', '조합원상태', '물품대분류', '물품중분류', '물품소분류',
       '물품명', '구매수량', '주소-구', '주소-동', '성별', '연령', '연령대', '구매금액', '구매매장',
       '반품_원거래일자', '구매시각'],
      dtype='object')

In [71]:
df_store['구매시각'] = df_store['구매시각'].apply(lambda t: t.hour*3600 + t.minute*60 + t.second)

In [72]:
df_store['구매시각'][:5]

0    36240
1    36300
2    36300
3    36480
4    36540
Name: 구매시각, dtype: int64

In [47]:
df_store['요일'].unique()

array(['화', '수', '목', '금', '토', '일', '월'], dtype=object)

## 예제 가설 : 요일(항목 7), 방문 시간(연속형)
- 분석 내용: 요일(범주형)에 따른 방문 시간(연속형)의 차이가 있는지 분석
  + 귀무가설: 차이가 없다
  + 대립가설: 차이가 있다

### 정규성 검증
- 비정규분포임을 확인 -> 범주형이 7집단이니 kruskal test 시행


In [ ]:
# stats.anderson을 사용한 정규 분포 검정
result = stats.anderson(df_store['구매시각'].dropna())
print('Test Statistic: %.3f' % result.statistic)
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < cv:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따름 (검정 통계량 %.3f < 임계값 %.3f)' % (sl, result.statistic, cv))
    else:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 %.3f >= 임계값 %.3f)' % (sl, result.statistic, cv))

Test Statistic: 7433.429
15.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 7433.429 >= 임계값 0.576)
10.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 7433.429 >= 임계값 0.656)
5.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 7433.429 >= 임계값 0.787)
2.5% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 7433.429 >= 임계값 0.918)
1.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 7433.429 >= 임계값 1.092)


In [ ]:
# 필요 항목 확인
days_list = df_store['요일'].value_counts().index.to_list()
days_list

['월', '화', '금', '수', '목', '토', '일']

In [ ]:
series_list = []
for day in days_list :
  condition = f"요일 =='{day}'"
  series_time = df_store.query(condition)['구매시각'].dropna()
  series_list.append(series_time)

In [ ]:
len(series_list[0])

123802

In [ ]:
# 등분산 검증
import scipy.stats as stats
stats.kruskal(*series_list)

KruskalResult(statistic=5229.456099901222, pvalue=0.0)

### 분석 결과
- 통계 결론 : 대립가설이 채택: anderson을 이용한 정규성 검증을 통해 연속형 데이터인 구매시각이 비정규 분포임을 확인해, 범주형 3집단 이상일 시 시행하는 Kruskal test 시행. pvalue가 0.0으로 확인 됨.  
- 사실 결론 : 요일에 따른 방문 시간의 차이가 있음.

## 예제 가설 : 지역(항목 15), 평균 구매 금액(연속형)
- 분석 내용: 지역별 고객의 평균 구매 금액의 차이가 있는지 분석
  + 귀무가설: 동일하다
  + 대립가설: 동일하지 않다.(차이가 있다)

In [90]:
df_store['구매금액']

0         22207
1          4977
2          7083
3           766
4          4403
          ...  
705566     9189
705567     5169
705568     7466
705569     3542
705570     1914
Name: 구매금액, Length: 705571, dtype: int64

In [97]:
df_store = df_store.rename(columns={'주소-구': '주소_구'})
df_store = df_store.rename(columns={'주소-동': '주소_동'})

### 정규성 검정
- 비정규분포임을 확인 -> 범주형이 15집단이니 kruskal test 시행


In [123]:
# stats.anderson을 사용한 정규 분포 검정
result = stats.anderson(df_store['구매금액'].dropna())
print('Test Statistic: %.3f' % result.statistic)
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < cv:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따름 (검정 통계량 %.3f < 임계값 %.3f)' % (sl, result.statistic, cv))
    else:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 %.3f >= 임계값 %.3f)' % (sl, result.statistic, cv))

Test Statistic: 80887.237
15.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.576)
10.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.656)
5.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.787)
2.5% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.918)
1.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 1.092)


In [116]:
# 필요 항목 확인
states_list = df_store['주소_구'].value_counts().index.to_list()
states_list

['수지구',
 '기타',
 '기흥구',
 '분당구',
 '수원 영통구',
 '서울 강남구',
 '서울 송파구',
 '처인구',
 '화성',
 '광주',
 '중원구',
 '수원 장안구',
 '수정구',
 '수원 팔달구',
 '하남',
 '수원 권선구']

In [119]:
series_list = []
for state in states_list :
  condition = f"주소_구 =='{state}'"
  series_purchase = df_store.query(condition)['구매금액'].dropna()
  series_list.append(series_purchase)

In [121]:
# 등분산 검증
import scipy.stats as stats
stats.kruskal(*series_list)

KruskalResult(statistic=81.68134280443158, pvalue=3.436671774662079e-11)

In [122]:
round(3.436671774662079e-11)

0

In [126]:
# 실제 값 확인
import numpy as np

df_store['주소_구'].value_counts()
pivot_state = pd.pivot_table(data = df_store[['주소_구', '구매금액']].dropna()
               , index='주소_구'
               , values='구매금액'
               , aggfunc=np.mean)
pivot_state.reset_index(inplace=True)
pivot_state.sort_values(by='구매금액', ascending=False)

,주소_구,구매금액
10,수정구,13010.338624
15,화성,11791.748976
13,처인구,11708.362209
4,서울 강남구,11566.960517
6,수원 권선구,11329.338384
5,서울 송파구,11057.427613
3,분당구,11055.815864
12,중원구,11048.423575
7,수원 영통구,10979.417175
11,수지구,10913.668224


### 분석 결과
- 통계 결론 : 대립가설이 채택: anderson을 이용한 정규성 검증을 통해 연속형 데이터인 구매금액이 비정규 분포임을 확인해, 범주형 3집단 이상일 시 시행하는 Kruskal test 시행. pvalue가 0.0으로 확인 됨.  
- 사실 결론 : 지역별 고객의 평균 구매 금액이 동일하지 않음. 따라서 지역에 따라 평균 구매 금액이 달라짐